### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

import html

def remove_punctuation(text):
    import string
    # translator = str.maketrans('', '', string.punctuation)
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    return html.unescape(text).translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [ ]:
#b)
baby_df['review'] = baby_df['review'].apply(lambda x: '' if x != x else x)

#short test:
baby_df["review"][38] == baby_df["review"][38]

In [ ]:
#a)
baby_df['review'] = baby_df['review'].apply(remove_punctuation)

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

In [ ]:
#c)

baby_df.drop(baby_df[baby_df['rating'] == 3].index, inplace = True)

#short test:
sum(baby_df["rating"] == 3)

In [ ]:
#d)

baby_df['rating'] = baby_df['rating'].apply(lambda x: 1 if x > 3 else -1)

#short test:
sum(baby_df["rating"]**2 != 1)

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())


In [ ]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [ ]:
#a)
from sklearn.model_selection import train_test_split

baby_df_train, baby_df_test = train_test_split(baby_df, test_size=0.3)

In [ ]:
#b)
vectorizer = CountVectorizer(
    max_df=0.999,
    min_df=5,
    max_features=10_000
)

baby_df_train_vec = vectorizer.fit_transform(baby_df_train['review'])
baby_df_test_vec = vectorizer.transform(baby_df_test['review'])

print(vectorizer.get_feature_names())

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [ ]:
#a)
model = LogisticRegression(
    solver='newton-cg',
    multi_class='ovr'
    #, class_weight='balanced'
    #, random_state=42
)

model.fit(baby_df_train_vec, baby_df_train['rating'])

In [ ]:
#b)

result = list(zip(*model.coef_, vectorizer.get_feature_names()))
result.sort(key = lambda x: x[0])
result.reverse()

out = '{1:15s} ({0: .3f})'
for word in result[:10]:
    print(out.format(*word))
print('...')
for word in result[-10:]:
    print(out.format(*word))

#hint: model.coef_, vectorizer.get_feature_names()

## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [ ]:
#a)

baby_df_test_prediction = model.predict(baby_df_test_vec)


In [ ]:
#b)

baby_df_test_prediction_prob = model.predict_proba(baby_df_test_vec)

#hint: model.predict_proba()

In [ ]:
#c)

tmp = sorted(zip(baby_df_test_prediction_prob, baby_df_test['review']),
    key = lambda x: x[0][1]
)

for i in tmp[-5:]:
    print(i[0])
    print(i[1][:80])

print('...')

for i in tmp[:5][::-1]:
    print(i[0])
    print(i[1][:80])

#hint: use the results of b)

In [ ]:
#d)

tn, fp, fn, tp = confusion_matrix(baby_df_test['rating'], baby_df_test_prediction).ravel()

row_format = '{:>12}' * 3
print(row_format.format('', 'Actual -1', 'Actual  1'))
print(row_format.format('Pred -1', tn, fn))
print(row_format.format('Pred  1', fp, tp))
print()

# precision: tp / (tp + fp)
# recall: tp / (tp + fn) = tp / p

print(classification_report(baby_df_test['rating'], baby_df_test_prediction))
print(accuracy_score(baby_df_test['rating'], baby_df_test_prediction))

## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [ ]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']
len(significant_words)

In [ ]:
#a)

vectorizer_5 = CountVectorizer(
    vocabulary = significant_words
)

baby_df_train_vec_5 = vectorizer_5.fit_transform(baby_df_train['review'])
baby_df_test_vec_5 = vectorizer_5.transform(baby_df_test['review'])

model_5 = LogisticRegression(
    solver='newton-cg',
    multi_class='ovr'
    #, class_weight='balanced'
    #, random_state=42
)

model_5.fit(baby_df_train_vec_5, baby_df_train['rating'])

result_5 = list(zip(*model_5.coef_, vectorizer_5.get_feature_names()))
result_5.sort()
result_5.reverse()

for word in result_5[:10]:
    print(out.format(*word))
print('...')
for word in result_5[-10:]:
    print(out.format(*word))

print()

baby_df_test_prediction_5 = model_5.predict(baby_df_test_vec_5)

baby_df_test_prediction_prob_5 = model_5.predict_proba(baby_df_test_vec_5)

tmp_5 = sorted(zip(baby_df_test_prediction_prob_5, baby_df_test['review']),
    key = lambda x: x[0][1]
)

for i in tmp_5[-5:]:
    print(i[0])
    print(i[1][:80])

print('...')

for i in tmp_5[:5][::-1]:
    print(i[0])
    print(i[1][:80])

print()

tn_5, fp_5, fn_5, tp_5 = confusion_matrix(baby_df_test['rating'], baby_df_test_prediction_5).ravel()

print(row_format.format('', 'Actual -1', 'Actual  1'))
print(row_format.format('Pred -1', tn_5, fn_5))
print(row_format.format('Pred  1', fp_5, tp_5))
print()

print(classification_report(baby_df_test['rating'], baby_df_test_prediction_5))
print(accuracy_score(baby_df_test['rating'], baby_df_test_prediction_5))

In [ ]:
#b)
bar = list(zip(*result_5))
plt.bar(bar[1], bar[0], zorder = 5)
plt.grid(True, axis='x')
plt.xticks(rotation=90)
plt.gcf().set_size_inches(10,5)
plt.show()

In [ ]:
#c)

tn, fp, fn, tp = confusion_matrix(baby_df_test['rating'], baby_df_test_prediction).ravel()

row_format = '{:>12}' * 3
print(row_format.format('', 'Actual -1', 'Actual  1'))
print(row_format.format('Pred -1', tn, fn))
print(row_format.format('Pred  1', fp, tp))
print()

print(classification_report(baby_df_test['rating'], baby_df_test_prediction))
print(accuracy_score(baby_df_test['rating'], baby_df_test_prediction))

print()
test1 = CountVectorizer(max_df=0.999, min_df=5, max_features=10_000).fit_transform(baby_df_train['review'])
%timeit -r4 LogisticRegression(solver='newton-cg', multi_class='ovr').fit(test1, baby_df_train['rating'])

print()
print()

tn_5, fp_5, fn_5, tp_5 = confusion_matrix(baby_df_test['rating'], baby_df_test_prediction_5).ravel()

print(row_format.format('', 'Actual -1', 'Actual  1'))
print(row_format.format('Pred -1', tn_5, fn_5))
print(row_format.format('Pred  1', fp_5, tp_5))
print()

print(classification_report(baby_df_test['rating'], baby_df_test_prediction_5))
print(accuracy_score(baby_df_test['rating'], baby_df_test_prediction_5))

print()
test2 = CountVectorizer(vocabulary = significant_words).fit(baby_df_train['review'])
%timeit -r10 LogisticRegression(solver='newton-cg', multi_class='ovr').fit(test2, baby_df_train['rating'])

#hint: %time, %timeit